In [1]:
# Libraries
import requests
import json

In [7]:
def fetch_and_save_data(url, file_path):
    # Make the API call
    response = requests.get(url)

    # Check if the call was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Ensure Unicode characters are properly decoded
        decoded_data = json.loads(json.dumps(data, ensure_ascii=False))

        # Save the response in a JSON file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(decoded_data, f, indent=4, ensure_ascii=False)

        return decoded_data
    else:
        print("Error in the API call")
        return None

# URL of Mercado Libre API
url = "https://api.mercadolibre.com/sites/MLA/search?q=iphone"
file_path = 'Examples/example_iphone.json'

# Call the function
data = fetch_and_save_data(url, file_path)

In [8]:
# URL of Mercado Libre sites API
sites_url = "https://api.mercadolibre.com/sites"
file_path_countries = 'Examples/example_countries.json'

# Call the function
countries_data = fetch_and_save_data(sites_url, file_path_countries)


In [9]:
# URL of Mercado Libre categories API for Mexico
categories_url = "https://api.mercadolibre.com/sites/MLM/categories"
file_path_categories = 'Examples/example_categories.json'

# Call the function
categories_data = fetch_and_save_data(categories_url, file_path_categories)

In [10]:
def fetch_paginated_data(category, country, start_offset, end_offset, file_path):
    base_url = "https://api.mercadolibre.com/sites/"
    all_data = []
    
    for offset in range(start_offset, end_offset + 1, 50):
        # Construct the URL with the given parameters
        url = f"{base_url}{country}/search?category={category}&offset={offset}"
        
        # Make the API call
        response = requests.get(url)
        
        # Check if the call was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Append the results to the all_data list
            all_data.extend(data.get('results', []))
        else:
            print(f"Error in the API call at offset {offset}")
    
    # Save the aggregated data in a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, indent=4, ensure_ascii=False)
    
    return all_data

# Example usage
category = "MLA1000"
country = "MLA"
start_offset = 0
end_offset = 100
file_path = 'Examples/example_paginated_data.json'

# Call the function
paginated_data = fetch_paginated_data(category, country, start_offset, end_offset, file_path)

In [12]:
import pandas as pd

def fetch_paginated_data_to_df(category, country, start_offset, end_offset):
    base_url = "https://api.mercadolibre.com/sites/"
    all_data = []
    
    for offset in range(start_offset, end_offset + 1, 50):
        # Construct the URL with the given parameters
        url = f"{base_url}{country}/search?category={category}&offset={offset}"
        
        # Make the API call
        response = requests.get(url)
        
        # Check if the call was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Append the results to the all_data list
            all_data.extend(data.get('results', []))
        else:
            print(f"Error in the API call at offset {offset}")
    
    # Create a DataFrame from the aggregated data
    df = pd.DataFrame(all_data)
    
    return df

# Example usage
category = "MLM1648"
country = "MLM"
start_offset = 0
end_offset = 100

# Call the function
df_paginated_data = fetch_paginated_data_to_df(category, country, start_offset, end_offset)
print(df_paginated_data.head())

              id                                              title condition  \
0  MLM1943284629  Tablet Goodtel Android 13 G2 10.1 Pulgadas 64g...       new   
1  MLM3374476492  Computadora Lenovo Thinkcentre, Core I5-3ra, 8...       new   
2  MLM1988713103          Samsung Galaxy Tab A9 64gb 4gb Ram Silver       new   
3  MLM3414839820  Laptop Lenovo Ideapad Celeron 4gb + 128ssd + O...       new   
4   MLM931980768  Regulador De Voltaje 8 Contactos Koblenz Rs-14...       new   

                   thumbnail_id catalog_product_id listing_type_id  \
0  888901-MLU75379457955_032024        MLM26668800        gold_pro   
1  844932-MLM79222058202_092024               None        gold_pro   
2  936099-MLU74407176568_022024        MLM33068760    gold_special   
3  614866-MLA79583445747_092024        MLM27986873    gold_special   
4  652640-MLA45754006388_042021        MLM17328307    gold_special   

                                     sanitized_title  \
0  tablet-goodtel-android-13-g2-101-

#### Top Category Items Iter 

In [28]:
def fetch_paginated_data_to_df_filtered(category, country, start_offset, end_offset):
    base_url = "https://api.mercadolibre.com/sites/"
    all_data = []
    
    for offset in range(start_offset, end_offset + 1, 50):
        # Construct the URL with the given parameters
        url = f"{base_url}{country}/search?category={category}&offset={offset}"
        
        # Make the API call
        response = requests.get(url)
        
        # Check if the call was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the required fields
            for item in data.get('results', []):
                filtered_item = {
                    'id': item.get('id'),
                    'seller_id': item.get('seller_id'),
                    'price': item.get('price'),
                    'original_price': item.get('original_price'),
                    'available_quantity': item.get('available_quantity'),
                }
                all_data.append(filtered_item)
        else:
            print(f"Error in the API call at offset {offset}")
    
    # Create a DataFrame from the filtered data
    df_filtered = pd.DataFrame(all_data)
    
    return df_filtered

# Example usage
category = "MLM1648"
country = "MLM"
start_offset = 0
end_offset = 100

# Call the function
df_filtered_paginated_data = fetch_paginated_data_to_df_filtered(category, country, start_offset, end_offset)
print(df_filtered_paginated_data.head())

              id seller_id   price base_price  original_price  \
0  MLM2023658727      None  6299.0       None          6499.0   
1  MLM3374476492      None  2185.5       None          2350.0   
2  MLM1988713103      None  2899.0       None             NaN   
3  MLM3414839820      None  3699.0       None             NaN   
4   MLM931980768      None   349.0       None             NaN   

  initial_quantity  available_quantity date_created last_updated health  
0             None                   1         None         None   None  
1             None                   1         None         None   None  
2             None                 500         None         None   None  
3             None                 500         None         None   None  
4             None                 500         None         None   None  


In [13]:
df_paginated_data.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,sanitized_title,permalink,buying_mode,site_id,...,winner_item_id,catalog_listing,discounts,decorations,promotions,inventory_id,official_store_name,variation_id,variations_data,variation_filters
0,MLM1943284629,Tablet Goodtel Android 13 G2 10.1 Pulgadas 64g...,new,888901-MLU75379457955_032024,MLM26668800,gold_pro,tablet-goodtel-android-13-g2-101-pulgadas-64gb...,https://www.mercadolibre.com.mx/tablet-goodtel...,buy_it_now,MLM,...,None,True,None,None,None,GBBH38386,NaN,NaN,NaN,NaN
1,MLM3374476492,"Computadora Lenovo Thinkcentre, Core I5-3ra, 8...",new,844932-MLM79222058202_092024,None,gold_pro,computadora-lenovo-thinkcentre-core-i5-3ra-8gb...,https://articulo.mercadolibre.com.mx/MLM-33744...,buy_it_now,MLM,...,None,False,None,None,None,None,NaN,NaN,NaN,NaN
2,MLM1988713103,Samsung Galaxy Tab A9 64gb 4gb Ram Silver,new,936099-MLU74407176568_022024,MLM33068760,gold_special,samsung-galaxy-tab-a9-64gb-4gb-ram-silver,https://www.mercadolibre.com.mx/samsung-galaxy...,buy_it_now,MLM,...,None,True,None,None,None,None,PCEL,NaN,NaN,NaN
3,MLM3414839820,Laptop Lenovo Ideapad Celeron 4gb + 128ssd + O...,new,614866-MLA79583445747_092024,MLM27986873,gold_special,laptop-lenovo-ideapad-celeron-4gb-128ssd-offic...,https://www.mercadolibre.com.mx/laptop-lenovo-...,buy_it_now,MLM,...,None,True,None,None,None,GWMS22875,CSHOP,NaN,NaN,NaN
4,MLM931980768,Regulador De Voltaje 8 Contactos Koblenz Rs-14...,new,652640-MLA45754006388_042021,MLM17328307,gold_special,regulador-de-voltaje-8-contactos-koblenz-rs-14...,https://www.mercadolibre.com.mx/regulador-de-v...,buy_it_now,MLM,...,None,True,None,None,None,QOHZ48422,Virtual Demand,NaN,NaN,NaN


In [14]:
df_paginated_data.columns

Index(['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'sanitized_title', 'permalink', 'buying_mode',
       'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id',
       'order_backend', 'price', 'original_price', 'sale_price',
       'available_quantity', 'official_store_id', 'use_thumbnail_id',
       'accepts_mercadopago', 'shipping', 'stop_time', 'seller', 'address',
       'attributes', 'installments', 'winner_item_id', 'catalog_listing',
       'discounts', 'decorations', 'promotions', 'inventory_id',
       'official_store_name', 'variation_id', 'variations_data',
       'variation_filters'],
      dtype='object')

###     Seller

In [24]:
df_paginated_data.seller

0      {'id': 1371389236, 'nickname': 'LALA2023051001...
1      {'id': 276696598, 'nickname': 'ONEDIGITALMEXICO'}
2           {'id': 226900267, 'nickname': 'TIENDA PCEL'}
3              {'id': 1331658732, 'nickname': 'C SHOP.'}
4      {'id': 414112680, 'nickname': 'VIRTUALDEMAND.C...
                             ...                        
136         {'id': 97892065, 'nickname': 'SLIM_COMPANY'}
137         {'id': 1376844377, 'nickname': 'INMOTICALL'}
138             {'id': 701940328, 'nickname': 'QZY66MX'}
139    {'id': 1628900651, 'nickname': 'CNCUBOTONLINES...
140               {'id': 10265533, 'nickname': 'RYSCOM'}
Name: seller, Length: 141, dtype: object

In [25]:
import requests

def get_seller_info(seller_id):
    url = f"https://api.mercadolibre.com/users/{seller_id}"
    response = requests.get(url)

    if response.status_code == 200:
        seller_data = response.json()
        return seller_data
    else:
        print(f"Error: {response.status_code}")
        return None

# Example: Replace 'SELLER_ID' with a real ID
get_seller_info(1371389236)

{'id': 1371389236,
 'nickname': 'LALA20230510010741',
 'country_id': 'MX',
 'address': {'city': 'Cuauhtémoc', 'state': 'MX-DIF'},
 'user_type': 'normal',
 'site_id': 'MLM',
 'permalink': 'http://perfil.mercadolibre.com.mx/LALA20230510010741',
 'seller_reputation': {'level_id': '5_green',
  'power_seller_status': 'platinum',
  'transactions': {'period': 'historic', 'total': 16704}},
 'status': {'site_status': 'active'}}

#### Sellers iter

In [42]:
def get_seller_info(seller_ids):
    all_seller_data = []

    for seller_id in seller_ids:
        url = f"https://api.mercadolibre.com/users/{seller_id}"
        response = requests.get(url)

        if response.status_code == 200:
            seller_data = response.json()
            filtered_seller_data = {
                'id': seller_data.get('id'),
                'nickname': seller_data.get('nickname'),
                'user_type': seller_data.get('user_type'),
                'site_id': seller_data.get('site_id'),
                'permalink': seller_data.get('permalink'),
                'level_id': seller_data.get('seller_reputation', {}).get('level_id'),
                'power_seller_status': seller_data.get('seller_reputation', {}).get('power_seller_status'),
                'transactions_total': seller_data.get('seller_reputation', {}).get('transactions', {}).get('total'),
                'status': seller_data.get('status', {}).get('site_status')
            }
            all_seller_data.append(filtered_seller_data)
        else:
            print(f"Error: {response.status_code} for seller_id {seller_id}")

    # Convert the list of seller data to a DataFrame
    df_seller_info = pd.DataFrame(all_seller_data)
    
    return df_seller_info

# Example usage: Replace 'SELLER_ID_LIST' with a real list of seller IDs
seller_ids = [1371389236, 276696598, 226900267]
df_seller_info = get_seller_info(seller_ids)

if df_seller_info is not None:
    print(df_seller_info.head())

           id            nickname user_type site_id  \
0  1371389236  LALA20230510010741    normal     MLM   
1   276696598    ONEDIGITALMEXICO    normal     MLM   
2   226900267         TIENDA PCEL     brand     MLM   

                                           permalink level_id  \
0  http://perfil.mercadolibre.com.mx/LALA20230510...  5_green   
1  http://perfil.mercadolibre.com.mx/ONEDIGITALME...  5_green   
2      http://perfil.mercadolibre.com.mx/TIENDA+PCEL  5_green   

  power_seller_status  transactions_total  status  
0            platinum               16730  active  
1                gold                2212  active  
2            platinum               57989  active  


### Item metrics

In [27]:
import requests
import json

def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items/{item_id}"
    response = requests.get(url)

    if response.status_code == 200:
        item_data = response.json()
        
        # Guardar la respuesta en un archivo JSON
        with open('Examples/item_info.json', 'w', encoding='utf-8') as f:
            json.dump(item_data, f, indent=4, ensure_ascii=False)
        
        return item_data
    else:
        print(f"Error: {response.status_code}")
        return None

# Ejemplo: Reemplaza 'ITEM_ID' con un ID real
item_id = "MLA1465318508"
item_info = get_item_info(item_id)

if item_info:
    print(item_info)


{'id': 'MLA1465318508', 'site_id': 'MLA', 'title': '20 Pilas Aaa Energizer Max E92 Alcalina', 'seller_id': 656317421, 'category_id': 'MLA6384', 'official_store_id': None, 'price': 13214.88, 'base_price': 13214.88, 'original_price': 17388, 'currency_id': 'ARS', 'initial_quantity': 9654, 'sale_terms': [{'id': 'WARRANTY_TYPE', 'name': 'Tipo de garantía', 'value_id': '6150835', 'value_name': 'Sin garantía', 'value_struct': None, 'values': [{'id': '6150835', 'name': 'Sin garantía', 'struct': None}], 'value_type': 'list'}, {'id': 'INVOICE', 'name': 'Facturación', 'value_id': '6891885', 'value_name': 'Factura A', 'value_struct': None, 'values': [{'id': '6891885', 'name': 'Factura A', 'struct': None}], 'value_type': 'list'}, {'id': 'PURCHASE_MAX_QUANTITY', 'name': 'Cantidad máxima de compra', 'value_id': None, 'value_name': '15', 'value_struct': None, 'values': [{'id': None, 'name': '15', 'struct': None}], 'value_type': 'number'}], 'buying_mode': 'buy_it_now', 'listing_type_id': 'gold_special'

#### Items iter

In [40]:
import pandas as pd

def get_item_info(item_ids):
    all_response = []

    for item_id in item_ids:
        url = f"https://api.mercadolibre.com/items/{item_id}"
        response = requests.get(url)

        if response.status_code == 200:
            item_data = response.json()
            filtered_item = {
                'id': item_data.get('id'),
                'seller_id': item_data.get('seller_id'),
                'price': item_data.get('price'),
                'base_price': item_data.get('base_price'),
                'original_price': item_data.get('original_price'),
                'initial_quantity': item_data.get('initial_quantity'),
                'available_quantity': item_data.get('available_quantity'),
                'sold_quantity': item_data.get('sold_quantity'),
                'health': item_data.get('health'),
                'date_created': item_data.get('date_created'),
                'last_updated': item_data.get('last_updated')
            }
            all_response.append(filtered_item)
        else:
            print(f"Error: {response.status_code}")
    
    # Convert the response to a DataFrame
    df_item_info = pd.DataFrame(all_response)
        
    return df_item_info

# Example: Replace 'ITEM_ID' with a real ID
item_ids = ["MLA1465318508", "MLA1281087020"]
df_item_info = get_item_info(item_ids)

if df_item_info is not None:
    print(df_item_info.head())


              id  seller_id     price  base_price  original_price  \
0  MLA1465318508  656317421  13214.88    13214.88         17388.0   
1  MLA1281087020  262582139  46999.00    46999.00             NaN   

   initial_quantity available_quantity sold_quantity health  \
0              9654               None          None   None   
1              6297               None          None   None   

               date_created              last_updated  
0  2023-08-03T12:33:28.000Z  2024-12-10T02:33:54.517Z  
1  2022-12-13T17:23:33.000Z  2024-12-10T02:58:33.689Z  
